In [12]:
import pandas as pd
import os

pasta_csv = 'casos_dengue_rj'

# Dicionário de mapeamento das siglas para nomes de cidades e seus códigos
mapa_regioes = {
    'AR': {'cidade': 'Angra dos Reis', 'COD_CIDADE': 32},
    'RJ': {'cidade': 'Rio de Janeiro', 'COD_CIDADE': 33},
    'CG': {'cidade': 'Campos de Goytacazes', 'COD_CIDADE': 34},
    'VA': {'cidade': 'Valença', 'COD_CIDADE': 35}
}

consolidados = []

for arquivo in os.listdir(pasta_csv):
    if arquivo.endswith(".csv"):
        caminho_arquivo = os.path.join(pasta_csv, arquivo)
        
        try:
            df = pd.read_csv(caminho_arquivo, encoding='latin1')
        except UnicodeDecodeError:
            df = pd.read_csv(caminho_arquivo, encoding='utf-8')
        
        total = df['Casos notificados'].iloc[-1]
        
        # Extraindo o ano e a região do nome do arquivo
        nome, _ = os.path.splitext(arquivo)
        partes = nome.split('_')
        ano = partes[0][-4:]  # Pega os últimos 4 dígitos (ano)
        sigla_regiao = partes[1]  # Pega a sigla da região
        
        # Obtendo o nome da cidade e o ID da cidade a partir do dicionário
        regiao_info = mapa_regioes.get(sigla_regiao)
        cidade = regiao_info['cidade']
        cod_cidade = regiao_info['COD_CIDADE']
        
        consolidados.append({
            'Nome do Arquivo': arquivo,
            'Total': total,
            'Ano': ano,
            'Região': cidade,
            'COD_CIDADE': cod_cidade
        })

df_consolidado = pd.DataFrame(consolidados)

# Salvando o arquivo consolidado com as novas colunas
df_consolidado.to_csv('tabela_consolidada.csv', index=False)

print("Tabela consolidada salva como 'tabela_consolidada.csv'.")


Tabela consolidada salva como 'tabela_consolidada.csv'.


In [13]:
import pandas as pd

# Lendo a tabela consolidada
df_consolidado = pd.read_csv('tabela_consolidada.csv')

# Lendo a tabela de saúde
df_saude = pd.read_csv('2019-2023-casos de dengue.csv')

# Removendo espaços em branco nos nomes das colunas
df_saude.columns = df_saude.columns.str.strip()

# Convertendo os nomes das colunas para string
df_saude.columns = df_saude.columns.astype(str)

# Garantindo que as colunas de ano sejam do tipo numérico
for ano in ['2019', '2020', '2021', '2022', '2023']:
    if ano in df_saude.columns:
        df_saude[ano] = pd.to_numeric(df_saude[ano], errors='coerce')

# Iterando sobre as linhas da tabela consolidada
for _, row in df_consolidado.iterrows():
    ano = str(row['Ano'])  # Garantindo que 'Ano' seja tratado como string
    total = row['Total']
    cod_cidade = row['COD_CIDADE']
    
    # Debug: Verificando os valores
    print(f"Processando COD_CIDADE: {cod_cidade}, Ano: {ano}, Total: {total}")

    # Verificando se o COD_CIDADE existe na tabela de saúde
    if cod_cidade in df_saude['COD_CIDADE'].values:
        # Verificando se o ano é uma coluna válida
        if ano in df_saude.columns:
            # Atualizando a tabela de saúde com os valores correspondentes
            current_value = df_saude.loc[df_saude['COD_CIDADE'] == cod_cidade, ano].values[0]
            if pd.isna(current_value):
                # Se o valor atual for NaN, inicializa com 0
                current_value = 0
            
            new_value = current_value + total
            df_saude.loc[df_saude['COD_CIDADE'] == cod_cidade, ano] = new_value
            
            # Debug: Imprimindo o novo valor
            print(f"Atualizando {ano} para COD_CIDADE {cod_cidade}: novo valor é {new_value}")
        else:
            print(f"Ano '{ano}' não encontrado nas colunas da tabela de saúde.")
    else:
        print(f"COD_CIDADE '{cod_cidade}' não encontrado na tabela de saúde.")

# Salvando a tabela de saúde atualizada
df_saude.to_csv('tabela_saude_atualizada.csv', index=False)

print("Tabela de saúde atualizada salva como 'tabela_saude_atualizada.csv'.")


Processando COD_CIDADE: 32, Ano: 2019, Total: 1764
Atualizando 2019 para COD_CIDADE 32: novo valor é 1764
Processando COD_CIDADE: 34, Ano: 2019, Total: 177
Atualizando 2019 para COD_CIDADE 34: novo valor é 177
Processando COD_CIDADE: 33, Ano: 2019, Total: 22046
Atualizando 2019 para COD_CIDADE 33: novo valor é 22046
Processando COD_CIDADE: 35, Ano: 2019, Total: 190
Atualizando 2019 para COD_CIDADE 35: novo valor é 190
Processando COD_CIDADE: 32, Ano: 2020, Total: 187
Atualizando 2020 para COD_CIDADE 32: novo valor é 187
Processando COD_CIDADE: 34, Ano: 2020, Total: 18
Atualizando 2020 para COD_CIDADE 34: novo valor é 18
Processando COD_CIDADE: 33, Ano: 2020, Total: 1536
Atualizando 2020 para COD_CIDADE 33: novo valor é 1536
Processando COD_CIDADE: 35, Ano: 2020, Total: 104
Atualizando 2020 para COD_CIDADE 35: novo valor é 104
Processando COD_CIDADE: 32, Ano: 2021, Total: 104
Atualizando 2021 para COD_CIDADE 32: novo valor é 104
Processando COD_CIDADE: 34, Ano: 2021, Total: 94
Atualizan